# 关系标签
+ NA	0
+ 人物关系/亲属关系/配偶/丈夫/现夫	1
+ 人物关系/亲属关系/配偶/丈夫/前夫	2
+ 人物关系/亲属关系/配偶/丈夫/未婚夫	3
+ 人物关系/亲属关系/配偶/妻子/现妻	4
+ 人物关系/亲属关系/配偶/妻子/前妻	5
+ 人物关系/亲属关系/配偶/妻子/未婚妻	6
+ 人物关系/亲属关系/血亲/自然血亲/祖父母/爷爷	7
+ 人物关系/亲属关系/血亲/自然血亲/祖父母/奶奶	8
+ 人物关系/亲属关系/血亲/自然血亲/外祖父母/外公	9
+ 人物关系/亲属关系/血亲/自然血亲/父母/生父	10
+ 人物关系/亲属关系/血亲/自然血亲/父母/生母	11
+ 人物关系/亲属关系/血亲/自然血亲/子女/儿子	12
+ 人物关系/亲属关系/血亲/自然血亲/子女/女儿	13
+ 人物关系/亲属关系/血亲/自然血亲/孙子女/孙子	14 
+ 人物关系/亲属关系/血亲/自然血亲/孙子女/孙女	15
+ 人物关系/亲属关系/血亲/自然血亲/兄弟姐妹/哥哥	16
+ 人物关系/亲属关系/血亲/自然血亲/兄弟姐妹/弟弟	17
+ 人物关系/亲属关系/血亲/自然血亲/兄弟姐妹/姐姐	18
+ 人物关系/亲属关系/血亲/自然血亲/兄弟姐妹/妹妹	19
+ 人物关系/亲属关系/血亲/自然血亲/叔伯姑舅姨/叔伯	20
+ 人物关系/亲属关系/血亲/自然血亲/叔伯姑舅姨/舅舅	21
+ 人物关系/亲属关系/血亲/自然血亲/叔伯姑舅姨/姑妈	22
+ 人物关系/亲属关系/血亲/自然血亲/侄甥/侄子	23
+ 人物关系/亲属关系/血亲/自然血亲/侄甥/侄女	24
+ 人物关系/亲属关系/姻亲/血亲的配偶/子女的配偶/儿媳	25
+ 人物关系/亲属关系/姻亲/血亲的配偶/子女的配偶/女婿	26
+ 人物关系/亲属关系/姻亲/血亲的配偶/兄弟姐妹的配偶/嫂子	27
+ 人物关系/亲属关系/姻亲/配偶的血亲/配偶的父母/公公	28
+ 人物关系/亲属关系/姻亲/配偶的血亲/配偶的父母/岳父	29
+ 人物关系/社交关系/友谊关系/朋友	30
+ 人物关系/社交关系/感情关系/喜欢	31
+ 人物关系/社交关系/感情关系/恋人	32
+ 人物关系/师生关系/老师	33
+ 人物关系/师生关系/学生	34

In [1]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import torch.nn as nn
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn import preprocessing
from random import shuffle
BATCH_SIZE = 512
EPOCHS = 401
LABELNUMS = 36
SENTEN_LEN = 50
CLASSIFY_RATE = 0.5

In [2]:
def make_one_hot(data1):
    return (np.arange(LABELNUMS)==data1[:,None]).astype(np.integer)

In [3]:
PATH = "/Users/denhiroshi/Downloads/model_data/"

In [4]:
train_X = torch.from_numpy(np.load(PATH+"train_Sentense_np.npy").astype("float32"))
train_Y = np.load(PATH+"train_Relation_np.npy").astype("int32")
dev_X = torch.from_numpy(np.load(PATH+"dev_Sentense_np.npy").astype("float32"))
dev_Y = np.load(PATH+"dev_Relation_np.npy").astype("int32")

In [5]:
train_Y = train_Y.tolist()
dev_Y = dev_Y.tolist()

In [6]:
# OneHot
train_ontHot_y = []
dev_ontHot_y = []
for trainY in train_Y:
    tmp = np.eye(LABELNUMS)[trainY]
    tmp = tmp.sum(axis = 0).astype("int32")
    tmp = tmp[:-1]
    train_ontHot_y.append(tmp)
train_ontHot_y = torch.Tensor(train_ontHot_y)
print(train_ontHot_y.shape)
for devY in dev_Y:
    tmp = np.eye(LABELNUMS)[devY]
    tmp = tmp.sum(axis = 0).astype("int32")
    tmp = tmp[:-1]
    dev_ontHot_y.append(tmp)
dev_ontHot_y = torch.Tensor(dev_ontHot_y)
print(dev_ontHot_y.shape)

torch.Size([43501, 35])
torch.Size([38417, 35])


In [7]:
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class TEXTCNN(nn.Module):
    def __init__(self):
        super(TEXTCNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape (1, BATCH_SIZE, 200)
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=(1,202),
            ), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(50,1)), 
        )
        
        self.conv2 = nn.Sequential( 
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=(3,202), 
            ), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=(48,1)), 
        )
        
        self.conv3 = nn.Sequential( 
            nn.Conv2d(
                in_channels=1, 
                out_channels=16, 
                kernel_size=(5,202), 
                stride = 1,
            ), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(46,1)),
        )

    def forward(self, x):        
        x = x.reshape((-1,1,x.shape[1],x.shape[2]))
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x = torch.cat((x1,x2,x3),1)
        x = x.view(x.size(0), -1)   # 展平
        # torch.Size([512, 48])
        return x

In [8]:
class Bi_LSTM_max(torch.nn.Module):
    def __init__(self):
        super(Bi_LSTM_max,self).__init__()
        self.rnn = torch.nn.LSTM(
            input_size = 256,       # rnn input
            hidden_size = 128,     # rnn hidden unit
            num_layers = 3,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
            bidirectional=True
        )
#         self.maxpool = torch.nn.MaxPool2d((SENTEN_LEN,1))

    def forward(self,x):
        output,(h_n,c_n)=self.rnn(x.float())
#         output_in_last_timestep = self.maxpool(output)
#         output_in_last_timestep = output_in_last_timestep.reshape((output_in_last_timestep.shape[0],-1))
        # torch.Size([512, 256])
        return output

In [9]:
class Encoder(torch.nn.Module):
    def __init__(self):
        super(Encoder,self).__init__()
        self.rnn = torch.nn.LSTM(
            input_size = 202,       # rnn input
            hidden_size = 128,     # rnn hidden unit
            num_layers = 3,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
            bidirectional=True
        )
#         self.maxpool = torch.nn.MaxPool2d((SENTEN_LEN,1))

    def forward(self,x):
        output,(h_n,c_n)=self.rnn(x.float())
#         output_in_last_timestep = self.maxpool(output)
#         output_in_last_timestep = output_in_last_timestep.reshape((output_in_last_timestep.shape[0],-1))
        # torch.Size([512, 256])
        return output

In [10]:
class Bi_GRU_max(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.GRU(
            input_size = 256,       # rnn input
            hidden_size = 128,     # rnn hidden unit
            num_layers = 3,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
            bidirectional=True
        )
        self.maxpool = torch.nn.MaxPool2d((SENTEN_LEN,1))

    def forward(self,x):
        output,h_n=self.rnn(x.float())
        output_in_last_timestep = self.maxpool(output)
        output_in_last_timestep = output_in_last_timestep.reshape((output_in_last_timestep.shape[0],-1))
        # torch.Size([512, 256])
        return output_in_last_timestep

In [11]:
class PCNN(nn.Module):
    def __init__(self):
        super(PCNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape (1, 10, 200)
            nn.Conv2d(
                in_channels=1,      # input height
                out_channels=50,    # n_filters
                kernel_size=(1,256),      # filter size
            ),      # output shape (16, 53, 200)
            nn.ReLU(),    # activation
        )
        self.maxpool = nn.MaxPool2d(kernel_size=(SENTEN_LEN,1))   # 在 2x2 空间里向下采样, output shape (16, 53, 1)

    def forward(self, x, x_pos):
        pos = []
#         for i in range(x.shape[0]):
#             tmp1 = (x[i,:,-2] == 0).nonzero()[0]
#             tmp2 = (x[i,:,-1] == 0).nonzero()[0]
#             tmp = torch.cat((tmp1,tmp2))
#             pos.append(tmp)
        for i in range(x_pos.shape[0]):
            tmp1 = (x_pos[i,:,-2] == 0).nonzero()[0]
            tmp2 = (x_pos[i,:,-1] == 0).nonzero()[0]
            tmp = torch.cat((tmp1,tmp2))
            pos.append(tmp)
        pos = torch.stack(pos)
        x = x.reshape((-1,1,SENTEN_LEN,x.shape[-1]))
        x = self.conv1(x)
        
        x1 = torch.zeros(x.shape)
        x2 = torch.zeros(x.shape)
        x3 = torch.zeros(x.shape)
        
        min_x1 = torch.ones(x.shape) * (-100)
        min_x2 = torch.ones(x.shape) * (-100)
        min_x3 = torch.ones(x.shape) * (-100)
        
        for i in range(pos.shape[0]):
            x1[i, :, :pos[i][0], :]
            min_x1[i, :, :pos[i][0], :]  = 0.0
            x2[i, :, pos[i][0]:pos[i][1], :]  = 1.0
            min_x2[i, :, pos[i][0]:pos[i][1], :] = 0.0
            x3[i, :, pos[i][1]:, :] = 1.0
            min_x3[i, :, pos[i][1]:, :] = 0.0
            
         # 卷积结果*mask得到分段后的卷积向量
        x1 *= x
        x2 *= x
        x3 *= x
        
        # 将无用部分赋予最小值，避免影响maxpool操作
        x1 += min_x1
        x2 += min_x2
        x3 += min_x3
        
        # 分段池化
        x1 = self.maxpool(x1)
        x2 = self.maxpool(x2)
        x3 = self.maxpool(x3)
        
        # 展平
        x1 = x1.view(x1.size(0), -1)
        x2 = x2.view(x2.size(0), -1)
        x3 = x3.view(x3.size(0), -1)
        
        x = torch.cat((x1,x2,x3),dim=1)
        return x

In [12]:
class Global_TEXTCNN(nn.Module):
    def __init__(self):
        super(Global_TEXTCNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape (1, BATCH_SIZE, 200)
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=(1,256),
            ), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(50,1)), 
        )
        
        self.conv2 = nn.Sequential( 
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=(3,256), 
            ), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=(48,1)), 
        )
        
        self.conv3 = nn.Sequential( 
            nn.Conv2d(
                in_channels=1, 
                out_channels=16, 
                kernel_size=(5,256), 
                stride = 1,
            ), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(46,1)),
        )
        
        self.output = nn.Linear(48, LABELNUMS-1) 

    def forward(self, x):
        x = x.reshape((-1,1,x.shape[1],x.shape[2]))
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x = torch.cat((x1,x2,x3),1)
        x = x.view(x.size(0), -1)   # 展平
#         x = self.out(x)
        output = self.output(x)
        return output

# 模型示意图
![](rank.png)

In [13]:
from torch.nn import init
class Rank_model(nn.Module):
    def __init__(self):
        super(Rank_model, self).__init__()
        # Global_Model
        self.Global_Model = Global_TEXTCNN()
        
        # Encoder
        self.Encoder = Encoder()
        self.maxpool = torch.nn.MaxPool2d((SENTEN_LEN,1))
        
        # NA-人物关系 <NA=0>
        self.model1 = Bi_LSTM_max() 
        self.model1_Liner = nn.Linear(256,2)
        
        # 人物关系/亲属关系-社交关系-师生关系
        self.model2 = Bi_LSTM_max() 
        self.model2_Liner = nn.Linear(256,3)
        
        # 亲属关系/配偶-血亲-姻亲
        self.model3 = Bi_LSTM_max() 
        self.model3_Liner = nn.Linear(256,3)
        
        # 社交关系/友谊关系-感情关系  <友谊关系/朋友=30>
        self.model4 = Bi_LSTM_max()
        self.model4_Liner = nn.Linear(256,2)
        
        # 师生关系/老师-学生 <叶子结点使用全联接层输出33, 34>
        self.Teacher_Student_CNN = PCNN()
        self.Teacher_Student = nn.Linear(150,2)
        
        # 配偶/丈夫-妻子
        self.model5 = Bi_LSTM_max()
        self.model5_Liner = nn.Linear(256,2)
        
        # 血亲/自然血亲/祖父母-外祖父母-父母-子女-孙子女-兄弟姐妹-叔伯姑舅姨-侄甥 <外祖父母/外公=9>
        self.model6 = Bi_LSTM_max()
        self.model6_Liner = nn.Linear(256,8)
        
        # 姻亲/血亲的配偶-配偶的血亲
        self.model7 = Bi_LSTM_max()
        self.model7_Liner = nn.Linear(256,2)
        
        # 恋人-喜欢  <叶子结点使用全联接层输出 31, 32>
        self.Love_Like_CNN = PCNN()
        self.Love_Like = nn.Linear(150,2)
        
        # 丈夫/现夫-前夫-未婚夫 <叶子结点使用全联接层输出1, 2, 3>
        self.now_pre_un_man_CNN = PCNN()
        self.now_pre_un_man = nn.Linear(150,3)
        
        # 妻子/现妻-前妻-未婚妻 <叶子结点使用全联接层输出4, 5, 6>
        self.now_pre_un_woman_CNN = PCNN()
        self.now_pre_un_woman = nn.Linear(150,3)
        
        # 血亲的配偶/子女的配偶-兄弟姐妹的配偶 <兄弟姐妹的配偶/嫂子=27>
        self.model8 = Bi_LSTM_max()
        self.model8_Liner = nn.Linear(256,2)
        
        # 配偶的血亲/配偶的父母/公公-岳父 <叶子结点使用全联接层输出28, 29>
        self.father_in_law_CNN = PCNN()
        self.father_in_law = nn.Linear(150,2)
        
        # 祖父母/爷爷-奶奶 <叶子结点使用全联接层输出7, 8>
        self.grandparents_CNN = PCNN()
        self.grandparents = nn.Linear(150,2)
        
        # 父母/生父-生母 <叶子结点使用全联接层输出10, 11>
        self.parents_CNN = PCNN()
        self.parents = nn.Linear(150,2)
        
        # 子女/儿子-女儿 <叶子结点使用全联接层输出12, 13>
        self.son_daughter_CNN = PCNN()
        self.son_daughter = nn.Linear(150,2)
        
        # 孙子女/孙子-孙女  <叶子结点使用全联接层输出14, 15>
        self.grandson_granddaughter_CNN = PCNN()
        self.grandson_granddaughter = nn.Linear(150,2)
        
        # 兄弟姐妹/哥哥-弟弟-姐姐-妹妹  <叶子结点使用全联接层输出16, 17, 18, 19>
        self.brothers_sisters_CNN = PCNN()
        self.brothers_sisters = nn.Linear(150,4)
        
        # 叔伯姑舅姨/叔伯-舅舅-姑妈 <叶子结点使用全联接层输出20, 21, 22>
        self.uncles_aunts_CNN = PCNN()
        self.uncles_aunts = nn.Linear(150,3)
        
        # 侄甥/侄子-侄女 <叶子结点使用全联接层输出23, 24>
        self.nephew_niece_CNN = PCNN()
        self.nephew_niece = nn.Linear(150,2)
        
        # 子女的配偶/儿媳-女婿 <叶子结点使用全联接层输出25, 26>
        self.daughter_son_in_law_CNN = PCNN()
        self.daughter_son_in_law = nn.Linear(150,2)
        
#         self.loss_Weight = nn.Linear(21,1)
        self.sigmod = nn.Sigmoid()
    
        self.LAMDA = torch.autograd.Variable(torch.Tensor([0.5]) ,requires_grad=True)
    
    def forward(self, x):
        # Encode
        x_Encode = self.Encoder(x)
        
        # NA-人物关系 <NA=0>
        after_Model1 = self.model1(x_Encode)
        after_Model1_max = self.maxpool(after_Model1)
        after_Model1_max = after_Model1_max.reshape(after_Model1_max.shape[0],-1)
        after_classify_model1 = self.model1_Liner(after_Model1_max)
        y_NA = ((self.sigmod(after_classify_model1[:,0]) >= CLASSIFY_RATE).nonzero()) # 0
        
        # 人物关系/亲属关系-社交关系-师生关系
        model2_Input = after_Model1
        after_Model2 = self.model2(model2_Input)
        after_Model2_max = self.maxpool(after_Model2)
        after_Model2_max = after_Model2_max.reshape(after_Model2_max.shape[0],-1)

        # 亲属关系/配偶-血亲-姻亲
        model3_Input = after_Model2
        after_Model3 = self.model3(model3_Input)

        # 社交关系/友谊关系-感情关系  <友谊关系/朋友=30>
        model4_Input = after_Model2
        after_Model4 = self.model4(model4_Input)
        after_Model4_max = self.maxpool(after_Model4)
        after_Model4_max = after_Model4_max.reshape(after_Model4_max.shape[0],-1)
        after_classify_model4 = self.model4_Liner(after_Model4_max)
        y_Firend = ((self.sigmod(after_classify_model4[:, 0]) >= CLASSIFY_RATE).nonzero()) # 3

        # 师生关系/老师-学生 <叶子结点使用全联接层输出33, 34>
        Teacher_Student_Input = after_Model2
        Teacher_Student_Input = self.Teacher_Student_CNN(Teacher_Student_Input, x[:, -2:])
        after_classify_Teacher_Student = self.Teacher_Student(Teacher_Student_Input)
        y_Teacher = ((self.sigmod(after_classify_Teacher_Student[:, 0]) >= CLASSIFY_RATE).nonzero()) # 33
        y_Student = ((self.sigmod(after_classify_Teacher_Student[:, 1]) >= CLASSIFY_RATE).nonzero()) # 34

        # 配偶/丈夫-妻子
        model5_Input = after_Model3
        after_Model5 = self.model5(model5_Input)
        after_Model5_max = self.maxpool(after_Model5)
        after_Model5_max = after_Model5_max.reshape(after_Model5_max.shape[0],-1)
        after_classify_model5 = self.model5_Liner(after_Model5_max)

        # 血亲/自然血亲/祖父母-外祖父母-父母-子女-孙子女-兄弟姐妹-叔伯姑舅姨-侄甥 <外祖父母/外公=9>
        model6_Input = after_Model3
        after_Model6 = self.model6(model6_Input)
        after_Model6_max = self.maxpool(after_Model6)
        after_Model6_max = after_Model6_max.reshape(after_Model6_max.shape[0],-1)
        after_classify_model6 = self.model6_Liner(after_Model6_max)
        y_grandpa = ((self.sigmod(after_classify_model6[:, 1]) >= CLASSIFY_RATE).nonzero()) # 9

        # 姻亲/血亲的配偶-配偶的血亲
        model7_Input = after_Model3
        after_Model7 = self.model7(model7_Input)
        after_Model7_max = self.maxpool(after_Model7)
        after_Model7_max = after_Model7_max.reshape(after_Model7_max.shape[0],-1)
        after_classify_model7 = self.model7_Liner(after_Model7_max)

        # 感情关系/恋人-喜欢  <叶子结点使用全联接层输出31, 32>
        Love_Like_Input = after_Model4
        Love_Like_Input = self.Love_Like_CNN(Love_Like_Input, x[:, -2:])
        after_Love_Like = self.Love_Like(Love_Like_Input)
        y_Love = ((self.sigmod(after_Love_Like[:,0]) >= CLASSIFY_RATE).nonzero()) # 31
        y_Like = ((self.sigmod(after_Love_Like[:,1]) >= CLASSIFY_RATE).nonzero()) # 32

        # 丈夫/现夫-前夫-未婚夫 <叶子结点使用全联接层输出1, 2, 3>
        now_pre_un_man_Input = after_Model5
        now_pre_un_man_Input = self.now_pre_un_man_CNN(now_pre_un_man_Input, x[:, -2:])
        after_now_pre_un_man = self.now_pre_un_man(now_pre_un_man_Input)
        y_now_man = ((self.sigmod(after_now_pre_un_man[:, 0]) >= CLASSIFY_RATE).nonzero()) # 1
        y_pre_man = ((self.sigmod(after_now_pre_un_man[:, 1]) >= CLASSIFY_RATE).nonzero()) # 2
        y_un_man = ((self.sigmod(after_now_pre_un_man[:, 2]) >= CLASSIFY_RATE).nonzero()) # 3

        # 妻子/现妻-前妻-未婚妻 <叶子结点使用全联接层输出4, 5, 6>
        now_pre_un_woman_Input = after_Model5
        now_pre_un_woman_Input = self.now_pre_un_woman_CNN(now_pre_un_woman_Input, x[:, -2:])
        after_now_pre_un_woman = self.now_pre_un_woman(now_pre_un_man_Input)
        y_now_woman = ((self.sigmod(after_now_pre_un_woman[:, 0]) >= CLASSIFY_RATE).nonzero()) # 4
        y_pre_woman = ((self.sigmod(after_now_pre_un_woman[:, 1]) >= CLASSIFY_RATE).nonzero()) # 5
        y_un_woman = ((self.sigmod(after_now_pre_un_woman[:, 2]) >= CLASSIFY_RATE).nonzero()) # 6

        # 血亲的配偶/子女的配偶-兄弟姐妹的配偶 <兄弟姐妹的配偶/嫂子=27>
        model8_Input = after_Model7
        after_Model8 = self.model8(model8_Input)
        after_Model8_max = self.maxpool(after_Model8)
        after_Model8_max = after_Model8_max.reshape(after_Model8_max.shape[0],-1)
        after_classify_model8 = self.model8_Liner(after_Model8_max)
        y_sister_in_law = ((self.sigmod(after_classify_model8[:, 1]) >= CLASSIFY_RATE).nonzero()) # 27

        # 配偶的血亲/配偶的父母/公公-岳父 <叶子结点使用全联接层输出28, 29>
        father_in_law_Input = after_Model7
        father_in_law_Input = self.father_in_law_CNN(father_in_law_Input, x[:, -2:])
        after_father_in_law = self.father_in_law(father_in_law_Input)
        y_wife_father = ((self.sigmod(after_father_in_law[:, 0]) >= CLASSIFY_RATE).nonzero()) # 28
        y_husband_father = ((self.sigmod(after_father_in_law[:, 1]) >= CLASSIFY_RATE).nonzero()) # 29

        # 祖父母/爷爷-奶奶 <叶子结点使用全联接层输出7, 8>
        grandparents_Input = after_Model6
        grandparents_Input = self.grandparents_CNN(grandparents_Input, x[:, -2:])
        after_grandparents = self.grandparents(grandparents_Input)
        y_grandfather = ((self.sigmod(after_grandparents[:, 0]) >= CLASSIFY_RATE).nonzero()) # 7
        y_grandmother = ((self.sigmod(after_grandparents[:, 1]) >= CLASSIFY_RATE).nonzero()) # 8

        # 父母/生父-生母 <叶子结点使用全联接层输出10, 11>
        grandparents_Input = after_Model6
        grandparents_Input = self.grandparents_CNN(grandparents_Input, x[:, -2:])
        after_parents = self.parents(grandparents_Input)
        y_father = ((self.sigmod(after_parents[:, 0]) >= CLASSIFY_RATE).nonzero()) # 10
        y_mother = ((self.sigmod(after_parents[:, 1]) >= CLASSIFY_RATE).nonzero()) # 11

        # 子女/儿子-女儿 <叶子结点使用全联接层输出12, 13>
        son_daughter_Input = after_Model6
        son_daughter_Input = self.son_daughter_CNN(son_daughter_Input, x[:, -2:])
        after_son_daughter = self.son_daughter(son_daughter_Input)
        y_son = ((self.sigmod(after_son_daughter[:, 0]) >= CLASSIFY_RATE).nonzero()) # 12
        y_daughter = ((self.sigmod(after_son_daughter[:, 1]) >= CLASSIFY_RATE).nonzero()) # 13

        # 孙子女/孙子-孙女  <叶子结点使用全联接层输出14, 15>
        grandson_granddaughter_Input = after_Model6
        grandson_granddaughter_Input = self.grandson_granddaughter_CNN(grandson_granddaughter_Input, x[:,-2:])
        after_grandson_granddaughter  = self.son_daughter(grandson_granddaughter_Input)
        y_grandson = ((self.sigmod(after_grandson_granddaughter[:, 0]) >= CLASSIFY_RATE).nonzero()) # 14
        y_granddaughter = ((self.sigmod(after_grandson_granddaughter[:, 1]) >= CLASSIFY_RATE).nonzero()) # 15

        # 兄弟姐妹/哥哥-弟弟-姐姐-妹妹  <叶子结点使用全联接层输出16, 17, 18, 19>
        brothers_sisters_Input = after_Model6
        brothers_sisters_Input = self.brothers_sisters_CNN(brothers_sisters_Input, x[:, -2:])
        after_brothers_sisters  = self.brothers_sisters(brothers_sisters_Input)
        y_elder_brother = ((self.sigmod(after_brothers_sisters[:, 0]) >= CLASSIFY_RATE).nonzero()) # 16
        y_younger_brother  = ((self.sigmod(after_brothers_sisters[:, 1]) >= CLASSIFY_RATE).nonzero()) # 17
        y_elder_sister  = ((self.sigmod(after_brothers_sisters[:, 2]) >= CLASSIFY_RATE).nonzero()) # 18
        y_younger_sister  = ((self.sigmod(after_brothers_sisters[:, 3]) >= CLASSIFY_RATE).nonzero()) # 19

        # 叔伯姑舅姨/叔伯-舅舅-姑妈 <叶子结点使用全联接层输出20, 21, 22>
        uncles_aunts_Input = after_Model6
        uncles_aunts_Input = self.uncles_aunts_CNN(uncles_aunts_Input, x[:, -2:])
        after_uncles_aunts  = self.uncles_aunts(uncles_aunts_Input)
        y_father_uncles = ((self.sigmod(after_uncles_aunts[:, 0]) >= CLASSIFY_RATE).nonzero()) # 20
        y_mother_uncles  = ((self.sigmod(after_uncles_aunts[:, 1]) >= CLASSIFY_RATE).nonzero()) # 21
        y_aunts  = ((self.sigmod(after_uncles_aunts[:, 2]) >= CLASSIFY_RATE).nonzero()) # 22


        # 侄甥/侄子-侄女 <叶子结点使用全联接层输出23, 24>
        nephew_niece_Input = after_Model6
        nephew_niece_Input = self.nephew_niece_CNN(nephew_niece_Input, x[:, -2:])
        after_nephew_niece  = self.nephew_niece(nephew_niece_Input)
        y_nephew = ((self.sigmod(after_nephew_niece[:, 0]) >= CLASSIFY_RATE).nonzero()) # 23
        y_niece  = ((self.sigmod(after_nephew_niece[:, 1]) >= CLASSIFY_RATE).nonzero()) # 24

        # 子女的配偶/儿媳-女婿 <叶子结点使用全联接层输出25, 26>
        daughter_son_in_law_Input = after_Model8
        daughter_son_in_law_Input = self.daughter_son_in_law_CNN(daughter_son_in_law_Input, x[:, -2:])
        after_daughter_son_in_law = self.daughter_son_in_law(daughter_son_in_law_Input)
        y_daughter_in_law = ((self.sigmod(after_daughter_son_in_law[:, 0]) >= CLASSIFY_RATE).nonzero()) # 25
        y_son_inlaw = ((self.sigmod(after_daughter_son_in_law[:, 1]) >= CLASSIFY_RATE).nonzero()) # 26
        
        # Glable_Model
        Global_output = self.Global_Model(x_Encode)
        output = [
            y_NA, y_now_man, y_pre_man, y_un_man, y_now_woman, y_pre_woman, y_un_woman, y_grandfather, y_grandmother, 
            y_grandpa, y_father, y_mother, y_son, y_daughter, y_grandson, y_granddaughter, y_elder_brother, y_younger_brother, 
            y_elder_sister, y_younger_sister, y_father_uncles, y_mother_uncles, y_aunts, y_nephew, y_niece, y_daughter_in_law, 
            y_son_inlaw, y_sister_in_law, y_wife_father, y_husband_father, y_Firend, y_Love, y_Like, y_Teacher, y_Student
        ]
        
        use = [
            after_classify_model1[y_NA.reshape(-1).tolist()][:, 0], 
            after_now_pre_un_man[y_now_man.reshape(-1).tolist()][:, 0], 
            after_now_pre_un_man[y_pre_man.reshape(-1).tolist()][:, 1], 
            after_now_pre_un_man[y_un_man.reshape(-1).tolist()][:, 2],
            after_now_pre_un_woman[y_now_woman.reshape(-1).tolist()][:, 0], 
            after_now_pre_un_woman[y_pre_woman.reshape(-1).tolist()][:, 1], 
            after_now_pre_un_woman[y_un_woman.reshape(-1).tolist()][:, 2], 
            after_grandparents[y_grandfather.reshape(-1).tolist()][:, 0],
            after_grandparents[y_grandmother.reshape(-1).tolist()][:, 1], 
            after_classify_model6[y_grandpa.reshape(-1).tolist()][:, 1], 
            after_parents[y_father.reshape(-1).tolist()][:, 0],
            after_parents[y_mother.reshape(-1).tolist()][:, 1],
            after_son_daughter[y_son.reshape(-1).tolist()][:, 0],
            after_son_daughter[y_daughter.reshape(-1).tolist()][:, 1],
            after_grandson_granddaughter[y_grandson.reshape(-1).tolist()][:, 0],
            after_grandson_granddaughter[y_granddaughter.reshape(-1).tolist()][:, 1],
            after_brothers_sisters[y_elder_brother.reshape(-1).tolist()][:, 0],
            after_brothers_sisters[y_younger_brother.reshape(-1).tolist()][:, 1],
            after_brothers_sisters[y_elder_sister.reshape(-1).tolist()][:, 2],
            after_brothers_sisters[y_younger_sister.reshape(-1).tolist()][:, 3],
            after_uncles_aunts[y_father_uncles.reshape(-1).tolist()][:, 0],
            after_uncles_aunts[y_mother_uncles.reshape(-1).tolist()][:, 1],
            after_uncles_aunts[y_aunts.reshape(-1).tolist()][:, 2],
            after_nephew_niece[y_nephew.reshape(-1).tolist()][:, 0],
            after_nephew_niece[y_niece.reshape(-1).tolist()][:, 1],
            after_daughter_son_in_law[y_daughter_in_law.reshape(-1).tolist()][:, 0],
            after_daughter_son_in_law[y_son_inlaw.reshape(-1).tolist()][:, 1],
            after_classify_model8[y_sister_in_law.reshape(-1).tolist()][:, 1],
            after_father_in_law[y_wife_father.reshape(-1).tolist()][:, 0],
            after_father_in_law[y_husband_father.reshape(-1).tolist()][:, 1],
            after_classify_model4[y_Firend.reshape(-1).tolist()][:, 0],
            after_Love_Like[y_Love.reshape(-1).tolist()][:, 0],
            after_Love_Like[y_Like.reshape(-1).tolist()][:, 1],
            after_classify_Teacher_Student[y_Teacher.reshape(-1).tolist()][:, 0],
            after_classify_Teacher_Student[y_Student.reshape(-1).tolist()][:, 1],
        ]
        tmp = torch.zeros(Global_output.shape)
        # lamda * local + (1-lamda) * gobal
        for i, out in enumerate(output):
            if out.shape[0] == 0:
                continue
            concat = torch.LongTensor([i for _ in range(Global_output.shape[0])])
            index =  (out.reshape(-1), concat.reshape(-1))
            tmp.index_put_(index, use[i])
        y = tmp * self.LAMDA + (1-self.LAMDA) * Global_output
        return [tmp, y]

In [14]:
model = Rank_model()
if torch.cuda.is_available():
    print("To GPU")
    train_X = train_X.cuda()
    train_ontHot_y = train_ontHot_y.cuda()
    dev_X = dev_X.cuda()
    dev_ontHot_y = dev_ontHot_y.cuda()
    model = model.cuda()
else:
    print("Still CPU")

Still CPU


In [15]:
train = TensorDataset(train_X, train_ontHot_y)
dev = TensorDataset(dev_X, dev_ontHot_y)

In [16]:
train_loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 0, 
)

dev_loader = DataLoader(
    dataset = dev, 
    batch_size = BATCH_SIZE,
    shuffle = True, 
    num_workers = 0, 
)

In [21]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
loss_F = torch.nn.BCELoss()
sigmoid = nn.Sigmoid()
for epoch in range(1,EPOCHS): 
    start = time.time()
    losses = []
    #print("1")
    for step,(x,y) in enumerate(train_loader):
        optimizer.zero_grad()
        start = time.time()
        output, predict = model(x)
        loss = loss_F(sigmoid(predict), y)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.mean())
        break
    #print("2")
    with torch.no_grad():
        accs = []
        for step,(devx,devy) in enumerate(dev_loader):
            dev_out, dev_pred = model(devx)
            dev_pred = sigmoid(dev_pred)
            tmp = dev_pred > 0.5
            count = 0
            for i in range(tmp.shape[0]):
                if torch.sum(tmp.int()[i] == devy.int()[i]).tolist() == devy.shape[1]:
                    count+=1
            acc = count / devy.shape[0]
            accs.append(acc)
        print(f"{epoch}: accuracy:{sum(accs)/len(accs)} loss: {torch.mean(torch.stack(losses))} time: {time.time() - start}")
    #print("flag")
    #assert False

AssertionError: 